### RNAscope starfish recipe

This recipe details all the preparation and ingredients necessary to cook up a starfish analysis pipeline for RNAscope experiments. It was written for users at CZBiohub and assumes the user has access to the imagingDB at CZBiohub.

This recipe covers the following steps in cooking up a batch of starfish gumbo:

- Accessing image files loaded onto the imaging server at CZBiohub
- Creating starfish Experiment files from images accessed on the server
- Working with starfish Experiment files and viewing images in napari
- Image pre-processing for detecting flourescent spots
- Identifying spots and overlaying them with the raw images in napari
- Segmenting cells using starfish watershed algorithm
- Constructing codebooks that assign targets to spots
- Producing a target by cell matrix and other target statistics

For questions or to request access to the imagingDB, contact recipe author @ andrew.cote@czbiohub.org

#### Accessing image files loaded onto the imaging server at CZBiohub

To access images located on the server, you require login credentials for the database. These can be provided by Andrew Cote (andrew.cote@czbiohub.org), in addition to installing the requisite libraries (InSitu Toolkit @ https://github.com/czbiohub/InSituToolkit). 

The login credentials are a simple .json file which we use to authenticate your requests to the database. This can be stored locally on your laptop. 


In [1]:
### To start, we either need to know the dataset ids ahead of time, have a csv file with them (as might be 
# used to originaly upload the files from the microscope computer), or know the <id> 
# associated with the experiment files which we can then search in the database. 

# If we know the dataset id directly:
dataset_id = 'GW-2019-12-22-04-45-00-0001'

# Or even better, re-use the csv file we used on the microscope computer to upload the images 
import csv

list_of_datasets = []
with open('files_to_upload_example.csv') as csvfile:
    read_csv = csv.reader(csvfile, delimiter = ',')
    row_number = 0            # the top row of the csv file contains headers, which we want to ignore
    for row in read_csv:
        if row_number >= 1:
            list_of_datasets.append(row[0])
        row_number += 1
        

In [2]:
# OPTIONAL DETOUR (This can be skipped if you use the built in InSituToolkit functions).

# We can access all the metadata associated with the experiment by using database operations. 
# DatabaseOperations is a class in python that takes a unique dataset id in the constructor, and so must be re-made
# each time you want to query metadata for a different experiment

# A full tutorial for querying the database is given at 
# https://github.com/czbiohub/imagingDB/blob/master/notebooks/database_queries.ipynb

from imaging_db.database.db_operations import DatabaseOperations
import imaging_db.database.db_operations as db_ops
import imaging_db.utils.db_utils as db_utils

# Note: refer to your own db_credentials.json location stored locally
db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'  

dbops = DatabaseOperations(dataset_id)
credentials_str = db_utils.get_connection_str(db_credentials)
with db_ops.session_scope(credentials_str) as session:
    global_meta, frames_meta = dbops.get_frames_meta(session)
    
# global_meta and frames_meta now contained all the metadata associated with the whole experiment, and each frame

In [3]:
frames_meta

,channel_idx,slice_idx,time_idx,pos_idx,channel_name,file_name,sha256
0,0,0,0,0,DAPI,im_c000_z000_t000_p000.png,128f5f59822b2ffd21bbbc2d2334725cb9bbcf5e397a0e...
1,0,0,0,1,DAPI,im_c000_z000_t000_p001.png,2fd6dd6b7be297e3983e0c6ec4d56593d09053a0180aac...
2,0,0,0,2,DAPI,im_c000_z000_t000_p002.png,b209618f901315b1d0c2302d99614a29fd4b3d695cfdeb...
3,0,0,0,3,DAPI,im_c000_z000_t000_p003.png,1ce609447626423306f00bb1a8285576b632a700580c1e...
4,0,0,0,4,DAPI,im_c000_z000_t000_p004.png,2dc5411557f1715a2758b1fe616b98f2d51ef8f0a42fb4...
...,...,...,...,...,...,...,...
1645,4,10,0,25,Cy7,im_c004_z010_t000_p025.png,90087d8e4482bd333af89ccd5e643e31dd65cc093e983b...
1646,4,10,0,26,Cy7,im_c004_z010_t000_p026.png,560497689112cb4dc895409484909bca04a383391f95d8...
1647,4,10,0,27,Cy7,im_c004_z010_t000_p027.png,e2ecd12486be3b768fb296f4235ab646fe3a28f17b0e68...
1648,4,10,0,28,Cy7,im_c004_z010_t000_p028.png,a0b7ff61bd6310e3d2d7b66032e8e8f985fba2b158a9f0...


### Creating "starfish experiment" files from images accessed on the server

Once we are able to access the raw image files on the database, we'd like to create starfish 'Experiment' objects to simplify the later analysis. Each Experiment is a self-contained module that has all raw image data, as well as metadata. Subsequent analysis in this notebook is restricted to a single experiment, but can be generalized to many experiments as all Experiment objects have the same interface / methods. 

An Experiment object is essentially a series of .json files that contain metadata which reference the raw images on the database. Therefore they are fairly small in size and can be created and stored locally, ideally in a './experiments/' folder for ease of navigation.

In [4]:
# Create the directories to contain experiment files
import os
cwd = os.getcwd()

experiment_path = cwd + '/experiments/' + dataset_id + '/'

if not os.path.exists(experiment_path):
    os.mkdir(experiment_path)

In [5]:
# To create experiment files we need to find a few key pieces of metadata: positions, and channels 
# these could be retrieved through the above database queries but InSituToolkit exposes a few useful methods

from InSituToolkit.imaging_database import write_experiment, get_positions, get_channels, search_ids
from slicedimage import ImageFormat
db_credentials = '/Users/andrew.cote/Documents/db_credentials.json' 


# search the database for dataset id's that contain a certain string
set_of_datasets = search_ids(db_credentials, 'GW')

# find all the microscope positions for a dataset in the database
positions = get_positions(db_credentials, dataset_id)

# find the filters and channels used
# Note: it is good practice to inspect the channels variable manually to double check we are not mis-assigning channels
channels = get_channels(db_credentials, dataset_id)

nuc_channel = [channels[0]]
stain_channel = [channels[1]]
spot_channel = [channels[2], channels[3], channels[4]]

# Note: the dataset_id MUST be contained in a list. Multiple dataset ID's could be written to the same experiment
# if the channels are common among them. 

write_experiment(db_credentials, experiment_path, [dataset_id], 
                spot_channels = spot_channel, \
                nuc_channels = nuc_channel, \
                stain_channels = stain_channel, \
                positions = positions, \
                img_format = 'PNG')   # By default the InSituScope saves as .PNG files


In [6]:
# OPTIONAL DETOUR: For a larger number of experiments with the same <id>, we could generalize this to:
list_of_datasets = []
list_of_positions = []
list_of_channels = []
for dataset_id in search_ids(db_credentials, 'GW'):
    list_of_datasets.append(dataset_id)
    list_of_positions.append(get_positions(db_credentials, dataset_id))
    list_of_channels.append(get_channels(db_credentials, dataset_id))
    
# ... include the above code for creating directories and experiments (ommitted here for run-ability of this notebook)

### Working with Starfish Experiment files and viewing images in napari

Napari is a viewer that is built around manipulating high-dimensional image files, for example, the 5D image file from a starfish Experiment, where the dimensions are (Round, Channel, Z, X, Y). It also has convenient options for viewing spots, stains, and segmentation masks on top of raw image files. 

In [5]:
# note: the below command '%gui qt5' is only required in a jupyter notebook. In a standalone script, starfish.display 
# will open the napari window by default. 

%gui qt5
from starfish import Experiment, FieldOfView, display
import napari

exp = Experiment.from_json(experiment_path + 'experiment.json')

/opt/anaconda3/lib/python3.7/site-packages/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [6]:
# Experiment objects are dicts which hold all the image data for each microscope location, or fov. A likely use-case
# is to perform the same image processing technique to all fov's in an experiment. We can collect all keys as:

list_of_keys = []

for key in exp.keys():
    list_of_keys.append(key)

# a fov has multiple types of images depending on the data that was uploaded, for example, nuclei, or stain
# starfish by default returns a ImageStack Iterator object, which necessitates the call of 'next()' to 
# retrieve the actual image stack

fov = exp['fov_000']

sample_primary = next(fov.get_images('primary'))
sample_nuclei = next(fov.get_images('nuclei'))

In [7]:
print('Number of FOVS is ' + str(len(list_of_keys)))
print('The first two are ' + (str(list_of_keys[0:2])))

Number of FOVS is 30
The first two are ['fov_000', 'fov_001']


In [8]:
# To display multiple images in the same viewer, assign a variable name to the display(ImageStack) command

example_viewer = display(sample_primary)
display(sample_nuclei, viewer = example_viewer)

100%|██████████| 11/11 [00:03<00:00,  3.22it/s]


### Basic Image Processing in Starfish

Starfish has numerous built-in methods to do some simple image processing tasks, including:
- image registration (learning and applying transforms between successive imaging rounds)
- projection (reducing the dimensionality of the images, e.g. flattening the z-stack)
- filtering (high-pass or low-pass filtering to help isolate spots)

A more detailed list can be found here: https://spacetx-starfish.readthedocs.io/en/stable/api/image/index.html
For this example of an RNAscope workflow, since we are only imaging across 

In [7]:
# A common step is to project all images along the z-dimension by the maximum pixel value, as this captures the 
# information from the best in-focus plane. 

# The ImageStack.reduce() function can project along any dimension desired, ZPLANE, ROUNDS, etc. 

from starfish.types import Axes, Coordinates, Features, FunctionSource, TraceBuildingStrategies
from starfish.image import Filter, Segment


projected_z_stacks = []

for key in list_of_keys[0:1]:  # Change the indices to iterate over more fovs if desired
    img_raw = next(exp[key].get_images('primary'))
    img_proj_z = img_raw.reduce({Axes.ZPLANE}, func='max')   
    projected_z_stacks.append(img_proj_z)

100%|██████████| 33/33 [00:07<00:00,  4.16it/s]


In [8]:
# To isolate features that are 'spot like,' we can run a Gaussian high-pass then low-pass filter, i.e. a bandpass
# filter. This helps reduce cellular autoflourescence, which is usually low-frequency (i.e. broad and smeared out)

single_z_stack = projected_z_stacks[0]

# Bandpass filter on features that are Gaussian spot-like
ghp = Filter.GaussianHighPass(sigma=5)
high_passed = ghp.run(single_z_stack, verbose=True, in_place=False)

glp = Filter.GaussianLowPass(sigma=1)
band_passed = glp.run(high_passed, verbose=True, in_place=False)

100%|██████████| 3/3 [00:00<00:00, 89.10it/s]


<Image layer 'band-passed z stack' at 0x1c57656810>

In [11]:
# Compare the results of the bandpass filter. Notice that for one of the views available in napari, the background
# cell autofluourescence is greatly reduced. 
filtered_viewer = napari.Viewer()
filtered_viewer.add_image(single_z_stack.xarray.values, name='single z stack')
filtered_viewer.add_image(band_passed.xarray.values, name='band-passed z stack')


# Note that in this sample there is still large flourescent bands that we may want to eliminate. We can do so
# using a white top-hat filter, which will only let through features smaller than the radius specified. 

masking_radius = 2 # in pixels
filt = Filter.WhiteTophat(masking_radius, is_volume=False)
tophat_filtered = filt.run(band_passed, in_place=False)
filtered_viewer.add_image(tophat_filtered.xarray.values, name='top-hat filtered 2px')

<Image layer 'top-hat filtered 2px' at 0x1c6cff31d0>

### Spot-finding

After we've filtered the images to remove background fluourescence we can now identify spots corresponding to the flourescent molecules in the study. starfish makes this relatively straightfoward through a built-in spot-finding method. 

Further documentation can be found at https://spacetx-starfish.readthedocs.io/en/mcai-watershedtutorial/api/spots/index.html#module-starfish.spots.FindSpots


In [9]:
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import TraceBuildingStrategies
from starfish.core.spots.DecodeSpots.trace_builders import build_traces_sequential

# these parameters can be adjusted if spots are not found or large features are identified as spots that shouldnt be

bd = FindSpots.BlobDetector(
    min_sigma = 1,
    max_sigma = 10,
    num_sigma = 30,
    threshold = 0.01,
    measurement_type = 'mean')

spots = bd.run(image_stack = band_passed)

# viewing spots directly is not part of the usual starfish analysis procedure,
# therefore we need to use build_traces_sequential to covert the spots into an intensity table
# which is viewable in the native display. 
intensity_table = build_traces_sequential(spots)
viewer = display(stack = single_z_stack, spots=intensity_table)


### Segmenting cells using starfish watershed algorithm

Watershed algorithm is a method of segmenting cells based on changes in pixel intensity, by grouping pixels into 'basins,' taking as a starting point the nuclei of each cell. A more thorough explanation of the method as used by starfish can be found here:https://spacetx-starfish.readthedocs.io/en/mcai-watershedtutorial/creating_an_image_processing_pipeline/tutorials/exec_watershed_segmentation.html


In [20]:
from starfish.image import Segment

nuclei = next(fov.get_images("nuclei"))
stain = next(fov.get_images("stain"))
primary = next(fov.get_images(FieldOfView.PRIMARY_IMAGES))

In [ ]:
nuclei = nuclei.reduce({Axes.ZPLANE}, func='max')
primary = primary.reduce({Axes.ZPLANE}, func='max')
stain = stain.reduce({Axes.ZPLANE}, func='max')

# NOTE: TopHat filtering is not strictly necessary for segmentation but is optional if desired
# for Watershed segmenting, it helps to apply a TopHat filter to more clearly isolate the nuclei against background
# fluorescence, and also eliminate blur from out-of-focus plane. (Note we still projected along Z-axis to capture cells
# in different planes of focus, but wish to eliminate the out-of-focus components)

masking_radius = 15
filt = filt = Filter.WhiteTophat(masking_radius, is_volume=False)
nuclei_filt = filt.run(nuclei, in_place=False)
stain_filt  = filt.run(stain,  in_place=False)

# Segmenting is a process that can be fine-tuned by varying the filter masking_radius and thresholds below. It is smart
# to compare results in the napari viewer and iterate to get the desired result. 

dapi_thresh = 0.18
stain_thresh = 0.22
min_dist = 7

# Lets try two segmentations with different parameters and see how they compare
seg = Segment.Watershed(
    nuclei_threshold = dapi_thresh,
    input_threshold  = stain_thresh,
    min_distance     = min_dist)

seg2 = Segment.Watershed(
    nuclei_threshold = dapi_thresh*3/2,
    input_threshold  = stain_thresh*2/3,
    min_distance     = min_dist-2)  # min distance must be specified in pixels, so no floats!

mask = seg.run(stain_filt, nuclei_filt)
mask2 = seg2.run(stain_filt, nuclei_filt)

100%|██████████| 1/1 [00:00<00:00, 79.43it/s]


In [25]:
# another way of constructing a napari viewer and comparing results, assigning labels for ease of keeping track

disp = napari.Viewer()

disp.add_image(mask.to_label_image().xarray.values, name='mask')
disp.add_image(mask2.to_label_image().xarray.values, name='mask2')
disp.add_image(nuclei_filt.xarray.values, name='nuclei', colormap='red')

# Observe that the second mask has much sparser and far fewer nuclei than the first mask,
# as a result of lowering the input_threshold parameter.

/opt/anaconda3/lib/python3.7/site-packages/starfish/core/morphology/object/binary_mask/expand.py:50: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  fill_value_array = result_array[selector]
/opt/anaconda3/lib/python3.7/site-packages/starfish/core/morphology/object/binary_mask/expand.py:52: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result_array[selector] = fill_value_array
/opt/anaconda3/lib/python3.7/site-packages/starfish/core/morphology/object/binary_mask/expand.py:50: FutureWarning: Using a non-tuple sequence for multidimensional in

<Image layer 'nuclei' at 0x1ca6531a50>

### Codebook Construction

We now assign targets to the spots found in the previous section. For RNAscope applications, each target flouresces in just one Round/Channel pair, in contrast to in-situ sequencing in which a single target will have different channels over multiple rounds. 

In [12]:
from starfish import Codebook
from starfish.types import Axes, Coordinates, Features

# RNAscope codebooks should only have one round value for each target gene, 
# as they are imaged in a single round


codebook_RNAscope = [
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "example_gene1"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "example_gene2"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "example_gene3"
      },
  ]


# we finish the codebook construction by calling the constructor for the starfish Codebook object. 
codebook = Codebook.from_code_array(codebook_RNAscope)

### Assigning spots to cells

With a codebook of target genes, a segmented image of the cells/nuclei, and a filtered image of spots, we are now ready to bring everything together to produce a cell x gene count matrix. 

First we decode the spots according to the codebook and a chosen trace building strategy, which translates from a items which are one or more (round, channel) pairs and converts them to targets in a codebook. 

Then we assign these targets to the cells segmented previously, and output the desired cell x gene table 

In [17]:
# for decoding spots, there are multiple different trace-building strategies which inform
# how starfish assigns the targets to given spots. For a single round of spots
# such as RNAscope, SEQUENTIAL is the best strategy and others may throw errors. 

spot_decoder = DecodeSpots.PerRoundMaxChannel(codebook = codebook,
                                             trace_building_strategy = TraceBuildingStrategies.SEQUENTIAL)

decoded_intensities = spot_decoder.run(spots = spots)

In [ ]:
al = AssignTargets.Label()
labeled = al.run(mask, decoded_intensities)
cg = labeled.to_expression_matrix()
cg